In [ ]:
#augmentation
import json
import os
import cv2
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt

def load_via_data(path):
    if os.path.exists(path):
        with open(path, 'r') as f:
            return json.load(f)
    else:
        return {}

# VIA JSON 파일 경로
via_json_path = r"C:\Users\LMK\Desktop\12\set_1\train\via.json"
output_dir = r"C:\Users\LMK\Desktop\12\set_1\train\transformed"
os.makedirs(output_dir, exist_ok=True)

# JSON 파일 읽기
via_data = load_via_data(via_json_path)

# 데이터 파싱
annotations = []
for key, value in via_data.items():
    filename = value['filename']
    regions = value['regions']
    polygons = []
    labels = []
    for region_key, region_value in regions.items():
        shape_attributes = region_value['shape_attributes']
        all_points_x = shape_attributes['all_points_x']
        all_points_y = shape_attributes['all_points_y']
        polygon = [(x, y) for x, y in zip(all_points_x, all_points_y)]
        polygons.append(polygon)
        labels.append(region_value['region_attributes']['name'])
        labels.append(region_value['region_attributes']['label'])
    annotations.append((filename, polygons, labels))

# 어그멘테이션 정의
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.HueSaturationValue(p=0.2),
    A.Resize(height=512, width=512, p=1.0),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

def clamp_keypoints(keypoints, width, height):
    clamped_keypoints = []
    for x, y in keypoints:
        clamped_x = max(0, min(x, width - 1))
        clamped_y = max(0, min(y, height - 1))
        clamped_keypoints.append((clamped_x, clamped_y))
    return clamped_keypoints

def save_transformed_data(data, output_path):
    with open(output_path, 'w') as f:
        json.dump(data, f)

# 변형된 이미지 저장 및 JSON 병합
merged_via_data = load_via_data(via_json_path)
all_transformed_data = {}

for iteration in range(4):
    print(f"Iteration {iteration + 1}")
    via_transformed_data = {}

    for filename, polygons, labels in annotations:
        image_path = f"C:\\Users\\LMK\\Desktop\\12\\set_1\\train\\{filename}"

        if not os.path.exists(image_path):
            print(f"Error: File '{image_path}' does not exist.")
            continue

        image = cv2.imread(image_path)
        if image is None:
            print(f"Error: Failed to read the image '{image_path}'.")
            continue

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        height, width, _ = image.shape

        keypoints = [point for polygon in polygons for point in polygon]
        clamped_keypoints = clamp_keypoints(keypoints, width, height)

        transformed = transform(image=image, keypoints=clamped_keypoints)
        transformed_image = transformed['image']
        transformed_keypoints = transformed['keypoints']

        transformed_polygons = []
        index = 0
        for polygon in polygons:
            num_points = len(polygon)
            transformed_polygon = transformed_keypoints[index:index+num_points]
            transformed_polygons.append(transformed_polygon)
            index += num_points

        transformed_filename = f"transformed_{iteration + 1}_{filename}"
        transformed_image_path = os.path.join(output_dir, transformed_filename)
        
        transformed_image_np = transformed_image.permute(1, 2, 0).cpu().numpy()
        transformed_image_np = np.clip(transformed_image_np * 255, 0, 255).astype(np.uint8)
        transformed_image_np = cv2.cvtColor(transformed_image_np, cv2.COLOR_RGB2BGR)
        cv2.imwrite(transformed_image_path, transformed_image_np)

        new_polygons = [{'all_points_x': [int(x) for x, y in polygon], 'all_points_y': [int(y) for x, y in polygon]} for polygon in transformed_polygons]

        new_key = f"{filename}_transformed_{iteration + 1}"
        via_transformed_data[new_key] = {
            'filename': transformed_filename,
            'regions': {str(i): {'shape_attributes': new_polygons[i], 'region_attributes': {'name': labels[i]}, 'region_attributes' : {'label':"license_plate"}} for i in range(len(new_polygons))}
        }

    # 각 반복마다 변환된 데이터를 병합 데이터에 추가
    all_transformed_data.update(via_transformed_data)

    # 각 반복마다 변환된 데이터 저장
    transformed_output_path = os.path.join(output_dir, f'via_transformed_{iteration + 1}.json')
    save_transformed_data(via_transformed_data, transformed_output_path)

    print(f"Total processed images in iteration {iteration + 1}: {len(via_transformed_data)}")

# 모든 변환된 데이터를 병합
merged_via_data.update(all_transformed_data)

merged_output_path = os.path.join(output_dir, 'via_merged.json')
save_transformed_data(merged_via_data, merged_output_path)


In [ ]:
#augmentation 후 이미지 시각화까지
import json
import os
import cv2
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt

# VIA JSON 파일 경로
via_json_path = r"C:\Users\LMK\Desktop\12\set_1\train\via.json"

# JSON 파일 읽기
with open(via_json_path, 'r') as f:
    via_data = json.load(f)

# 데이터 파싱
annotations = []
for key, value in via_data.items():
    filename = value['filename']
    regions = value['regions']
    polygons = []
    labels = []
    for region_key, region_value in regions.items():
        shape_attributes = region_value['shape_attributes']
        all_points_x = shape_attributes['all_points_x']
        all_points_y = shape_attributes['all_points_y']
        polygon = [(x, y) for x, y in zip(all_points_x, all_points_y)]
        polygons.append(polygon)
        labels.append(region_value['region_attributes']['name'])
    annotations.append((filename, polygons, labels))

# 어그멘테이션 정의
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.HueSaturationValue(p=0.2),
    A.Resize(height=512, width=512, p=1.0),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
], keypoint_params=A.KeypointParams(format='xy'))

def visualize(image, polygons):
    img = image.copy()
    for polygon in polygons:
        polygon = np.array(polygon, np.int32)
        polygon = polygon.reshape((-1, 1, 2))
        cv2.polylines(img, [polygon], isClosed=True, color=(0, 255, 0), thickness=2)
    plt.figure(figsize=(12, 12))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

# 변형된 이미지 저장 경로
output_dir = r"C:\Users\LMK\Desktop\12\set_1\train\transformed"
os.makedirs(output_dir, exist_ok=True)

# 예시로 첫 번째 이미지 처리
filename, polygons, labels = annotations[0]
image_path = f"C:\\Users\\LMK\\Desktop\\12\\set_1\\train\\{filename}"

# 파일 존재 여부 확인
if not os.path.exists(image_path):
    print(f"Error: File '{image_path}' does not exist.")
else:
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Failed to read the image '{image_path}'.")
    else:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        keypoints = [point for polygon in polygons for point in polygon]

        # 어그멘테이션 적용
        transformed = transform(image=image, keypoints=keypoints)
        transformed_image = transformed['image']
        transformed_keypoints = transformed['keypoints']

        # 변환된 키포인트를 폴리곤 형태로 재구성
        transformed_polygons = []
        index = 0
        for polygon in polygons:
            num_points = len(polygon)
            transformed_polygon = transformed_keypoints[index:index+num_points]
            transformed_polygons.append(transformed_polygon)
            index += num_points

        # 어그멘테이션 후 이미지와 폴리곤 시각화
        visualize(transformed_image.permute(1, 2, 0).numpy(), transformed_polygons)

        # 변형된 이미지 저장
        transformed_filename = f"transformed_{filename}"
        transformed_image_path = os.path.join(output_dir, transformed_filename)
        cv2.imwrite(transformed_image_path, transformed_image.permute(1, 2, 0).numpy()[:, :, ::-1] * 255)

        # 변환된 폴리곤을 원래 형식으로 변환
        new_polygons = [{'all_points_x': [int(x) for x, y in polygon], 'all_points_y': [int(y) for x, y in polygon]} for polygon in transformed_polygons]

        # 변환된 데이터 저장
        via_data[key]['filename'] = transformed_filename
        via_data[key]['regions'] = {str(i): {'shape_attributes': new_polygons[i], 'region_attributes': {'label': labels[i]}} for i in range(len(new_polygons))}

        # 변환된 JSON 파일 저장
        with open(r"C:\Users\LMK\Desktop\12\set_1\train\via_t.json", 'w') as f:
            json.dump(via_data, f)


In [ ]:
#섞기
import json
import os
import random

# JSON 파일 경로
via_json_path = r"C:\Users\LMK\Desktop\aug2\val\via_merged.json"
shuffled_json_path = r"C:\Users\LMK\Desktop\aug2\val\via_merged.json"

# JSON 파일 읽기
def load_via_data(path):
    if os.path.exists(path):
        with open(path, 'r') as f:
            return json.load(f)
    else:
        return {}

via_data = load_via_data(via_json_path)

# 데이터의 키를 리스트로 추출하여 무작위로 섞기
keys = list(via_data.keys())
random.shuffle(keys)

# 무작위로 섞인 순서로 새로운 사전 생성
shuffled_via_data = {key: via_data[key] for key in keys}

# 새로운 JSON 파일로 저장
with open(shuffled_json_path, 'w') as f:
    json.dump(shuffled_via_data, f, indent=4)

print(f"Shuffled JSON data saved to {shuffled_json_path}")


In [ ]:
import json
import os
import random

# JSON 파일 경로
via_json_path = r"C:\Users\LMK\Desktop\aug2\val\via_merged.json"
shuffled_json_path = r"C:\Users\LMK\Desktop\aug2\val\via_merged1.json"

# JSON 파일 읽기
def load_via_data(path):
    if os.path.exists(path):
        with open(path, 'r') as f:
            return json.load(f)
    else:
        return {}

via_data = load_via_data(via_json_path)

# 데이터 섞기 전 로그
print("Before shuffling:")
for key, value in via_data.items():
    for region_key, region_value in value['regions'].items():
        if 'label' in region_value['region_attributes']:
            print(f"{key}: {region_value['region_attributes']['label']}")

# 데이터의 키를 리스트로 추출하여 무작위로 섞기
keys = list(via_data.keys())
random.shuffle(keys)

# 무작위로 섞인 순서로 새로운 사전 생성
shuffled_via_data = {key: via_data[key] for key in keys}

# 데이터 섞기 후 로그
print("\nAfter shuffling:")
for key, value in shuffled_via_data.items():
    for region_key, region_value in value['regions'].items():
        if 'label' in region_value['region_attributes']:
            print(f"{key}: {region_value['region_attributes']['label']}")

# 새로운 JSON 파일로 저장
with open(shuffled_json_path, 'w') as f:
    json.dump(shuffled_via_data, f, indent=4)

print(f"Shuffled JSON data saved to {shuffled_json_path}")


In [ ]:
#train val 분할
import os
import random
import shutil

def get_random_samples(source_files, num_samples):
    return random.sample(source_files, num_samples)

def copy_files(files, source_dir, destination_dir):
    os.makedirs(destination_dir, exist_ok=True)
    for file in files:
        # 이미지 파일 복사
        shutil.copy(os.path.join(source_dir, file), os.path.join(destination_dir, file))
        # 동일한 이름의 라벨링 파일 복사
        label_file = os.path.splitext(file)[0] + '.xml'  # 예: image1.jpg -> image1.xml
        if os.path.exists(os.path.join(source_dir, label_file)):
            shutil.copy(os.path.join(source_dir, label_file), os.path.join(destination_dir, label_file))

def create_datasets(source_dir, output_base_dir, num_train_samples=380, num_val_samples=200, num_sets=1):
    # 이미지 파일만 선택 (확장자 .png 또는 .jpeg)
    source_files = [f for f in os.listdir(source_dir) if os.path.isfile(os.path.join(source_dir, f)) and (f.endswith('.png') or f.endswith('.jpeg') or f.endswith('.jpg'))]  # '.jpg' 확장자도 추가

    if len(source_files) < (num_train_samples + num_val_samples):
        raise ValueError("Source directory does not contain enough images to create the desired dataset sizes.")

    used_files = set()
    
    for i in range(1, num_sets + 1):
        available_files = list(set(source_files) - used_files)
        
        train_samples = get_random_samples(available_files, num_train_samples)
        used_files.update(train_samples)
        
        available_files = list(set(source_files) - used_files)
        val_samples = get_random_samples(available_files, num_val_samples)
        used_files.update(val_samples)
        
        output_train_dir = os.path.join(output_base_dir, f'set_{i}', 'train')
        output_val_dir = os.path.join(output_base_dir, f'set_{i}', 'val')
        
        copy_files(train_samples, source_dir, output_train_dir)
        copy_files(val_samples, source_dir, output_val_dir)
        
        print(f"Set {i} - Train samples: {len(train_samples)}")
        print(f"Set {i} - Val samples: {len(val_samples)}")

# 디렉토리 경로 설정
source_dir = r"C:\Users\LMK\Desktop\plate_new\images"
output_base_dir = r"C:\Users\LMK\Desktop\plate_new\plate"

# 데이터셋 생성
create_datasets(source_dir, output_base_dir)


In [ ]:
#json 내부 이미지와 라벨 매칭 여부
import json
import os

def load_json(path):
    """
    JSON 파일을 불러오는 함수입니다.
    """
    if os.path.exists(path):
        with open(path, 'r') as f:
            return json.load(f)
    else:
        return {}

def get_image_filenames(folder_path, extensions=['.jpg', '.jpeg', '.png']):
    """
    폴더 내의 이미지 파일 이름을 불러오는 함수입니다.
    """
    return [f for f in os.listdir(folder_path) if os.path.splitext(f)[1].lower() in extensions]

def check_images_in_json(json_data, image_filenames):
    """
    JSON 데이터에 모든 이미지 파일 정보가 포함되어 있는지 확인하는 함수입니다.
    """
    json_filenames = [entry['file_name'] for entry in json_data.values()]
    missing_files = [filename for filename in image_filenames if filename not in json_filenames]
    extra_files = [filename for filename in json_filenames if filename not in image_filenames]
    return missing_files, extra_files

# 경로 설정
json_path = r"C:\Users\LMK\Desktop\aug2\val\custom_var.json"
image_folder = r"C:\Users\LMK\Desktop\aug2\val"

# JSON 파일 및 이미지 파일 불러오기
json_data = load_json(json_path)
image_filenames = get_image_filenames(image_folder)

# JSON 데이터 확인
missing_files, extra_files = check_images_in_json(json_data, image_filenames)

# JSON 파일 내부의 file_name 개수 출력
json_image_filenames_count = sum(1 for entry in json_data.values() if 'file_name' in entry)

print(f"JSON 파일 내의 'file_name' 개수: {json_image_filenames_count}")

if missing_files:
    print("JSON에 포함되지 않은 이미지 파일들:")
    for file in missing_files:
        print(file)
else:
    print("모든 이미지 파일이 JSON에 포함되어 있습니다.")

if extra_files:
    print("폴더에 없는 JSON 파일들:")
    for file in extra_files:
        print(file)
else:
    print("모든 JSON 파일이 폴더에 존재합니다.")


In [ ]:
#이미지 파일 숫자 세기
import os

def count_image_files(directory, extensions=['.jpg', '.jpeg', '.png', '.bmp', '.gif']):
    count = 0
    for root, dirs, files in os.walk(directory):
        for file in files:
            if any(file.lower().endswith(ext) for ext in extensions):
                count += 1
    return count

# 이미지 파일이 있는 폴더 경로
image_folder_path = r"C:\Users\LMK\Desktop\plate_new\images"
# 이미지 파일 개수 세기
num_images = count_image_files(image_folder_path)
print(f"Total number of image files: {num_images}")


In [ ]:
#xml 수합후 json via로 병합
import os
import xml.etree.ElementTree as ET
import json

# Function to parse a single XML file and convert rect to polygon
def parse_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    filename = root.find('filename').text

    annotations = []
    for obj in root.findall('object'):
        name = obj.find('name').text
        xmin = int(obj.find('bndbox/xmin').text)
        ymin = int(obj.find('bndbox/ymin').text)
        xmax = int(obj.find('bndbox/xmax').text)
        ymax = int(obj.find('bndbox/ymax').text)

        # Convert rect to polygon
        annotations.append({
            "shape_attributes": {
                "name": "polygon",
                "all_points_x": [xmin, xmax, xmax, xmin],
                "all_points_y": [ymin, ymin, ymax, ymax]
            },
            "region_attributes": {
                "name": name,
                "label":"license_plate"
            }
        })
    
    return filename, annotations

# Function to convert parsed data to VIA format
def convert_to_via_format(parsed_data):
    via_data = {}
    for filename, annotations in parsed_data:
        via_data[filename] = {
            "filename": filename,
            "size": -1,  # Size is unknown
            "regions": {str(i): annotations[i] for i in range(len(annotations))},
            "file_attributes": {}
        }
    return via_data

# Directory containing XML files
xml_dir = r"C:\Users\LMK\Desktop\plate_new\plate\set_1\train"
# Parse all XML files in the directory
parsed_data = []
for xml_file in os.listdir(xml_dir):
    if xml_file.endswith('.xml'):
        parsed_data.append(parse_xml(os.path.join(xml_dir, xml_file)))

# Convert parsed data to VIA format
via_data = convert_to_via_format(parsed_data)

# Write VIA data to a JSON file in the xml_dir directory
via_json_file = os.path.join(xml_dir, 'via.json')
with open(via_json_file, 'w') as f:
    json.dump(via_data, f, indent=4)

print(f"VIA annotations have been saved to {via_json_file}")


In [ ]:
#데이터 삭제 후 라벨링 매칭여부
import os

def check_image_label_pairs(folder_path, image_extensions=['.jpg', '.jpeg', '.png'], label_extensions=['.json', '.xml']):
    """
    이미지와 라벨링 데이터가 동일한 이름으로 짝지어져 있는지 확인합니다.

    Args:
    folder_path (str): 파일을 검사할 폴더 경로
    image_extensions (list): 이미지 파일 확장자 리스트
    label_extensions (list): 라벨 파일 확장자 리스트

    Returns:
    dict: 짝지어지지 않은 이미지 및 라벨 파일 목록
    """
    # 폴더 내의 모든 파일 목록 가져오기
    files = os.listdir(folder_path)
    
    # 이미지 파일 및 라벨 파일 필터링
    image_files = [os.path.splitext(file)[0] for file in files if os.path.splitext(file)[1].lower() in image_extensions]
    label_files = [os.path.splitext(file)[0] for file in files if os.path.splitext(file)[1].lower() in label_extensions]
    
    # 이미지 파일 중 라벨이 없는 파일 찾기
    images_without_labels = [file for file in image_files if file not in label_files]
    
    # 라벨 파일 중 이미지가 없는 파일 찾기
    labels_without_images = [file for file in label_files if file not in image_files]
    
    return {
        "images_without_labels": images_without_labels,
        "labels_without_images": labels_without_images
    }

# 폴더 경로 지정
folder_path = r"C:\Users\LMK\Desktop\plate_new\plate\set_1\val"

# 함수 호출
unpaired_files = check_image_label_pairs(folder_path)

if unpaired_files["images_without_labels"]:
    print("라벨이 없는 이미지 파일:")
    for file in unpaired_files["images_without_labels"]:
        print(f"{file}")
else:
    print("라벨이 없는 이미지 파일이 없습니다.")

if unpaired_files["labels_without_images"]:
    print("이미지가 없는 라벨 파일:")
    for file in unpaired_files["labels_without_images"]:
        print(f"{file}")
else:
    print("이미지가 없는 라벨 파일이 없습니다.")
